## Setup

In [2]:
#imports
import os, sys, json, datetime, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time

from tqdm import tqdm
from functools import reduce
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown
from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.defined_functions import get_recent_file
# %%
os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files')

AL 10
AK 11
AZ 12
AR 13
CA 14
CO 15
CT 16
DE 17
FL 18
GA 19
HI 20
ID 21
IL 22
IN 23
IA 24
KS 25
KY 26
LA 27
ME 28
MD 29
MA 30
MI 31
MN 32
MS 33
MO 34
MT 35
NE 36
NV 37
NH 38
NJ 39
NM 40
NY 41
NC 42
ND 43
OH 44
OK 45
OR 46
PA 47
RI 48
SC 49
SD 50
TN 51
TX 52
UT 53
VT 54
VA 55
WA 56
WV 57
WI 58
WY 59
DC 60


In [3]:
#Get lookup Data
key_path = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors'
legislators_df = get_recent_file("*.csv", key_path)
# legislators_df = pd.read_csv(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors\leg_lookup_df.csv')

#grab scores data
dir_path = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files'
activity_file = get_recent_file("activity*.csv", dir_path)
influence_file = get_recent_file("*infl*.csv", dir_path)

#make dfs
legislators_df = pd.read_csv(legislators_df)
activities_df = pd.read_csv(activity_file)
influence_df = pd.read_csv(influence_file)

#testing print statement
all_dfs = [legislators_df,activities_df,influence_df]
for all in all_dfs:
    print(*all.columns, sep = ',')

legislators_df

full_pk,primary_key,district_code,state_abbreviation,district_string,chamber,district,state_code,chamber_code,seat_num
full_pk,primary_key,first_name,last_name,activities_score,events
full_pk,first_name,last_name,party,influence_score


,full_pk,primary_key,district_code,state_abbreviation,district_string,chamber,district,state_code,chamber_code,seat_num
0,10100100,101001,1,AL,Alabama State Senate District 1,Senate,1,10,1,0
1,10100200,101002,2,AL,Alabama State Senate District 2,Senate,2,10,1,0
2,10100300,101003,3,AL,Alabama State Senate District 3,Senate,3,10,1,0
3,10100400,101004,4,AL,Alabama State Senate District 4,Senate,4,10,1,0
4,10100500,101005,5,AL,Alabama State Senate District 5,Senate,5,10,1,0
...,...,...,...,...,...,...,...,...,...,...
1989,57101502,571015,15,WV,West Virginia State Senate District 15,Senate,15,57,1,2
1990,57101601,571016,16,WV,West Virginia State Senate District 16,Senate,16,57,1,1
1991,57101602,571016,16,WV,West Virginia State Senate District 16,Senate,16,57,1,2
1992,57101701,571017,17,WV,West Virginia State Senate District 17,Senate,17,57,1,1


In [4]:
#narrow down scores dfs
# activities_df = activities_df.loc[:,['full_pk', 'activities_score']]
# influence_df = influence_df.loc[:,['full_pk', 'influence_score']]

In [5]:
#merging
#merge influence scores
first_merge = pd.merge(legislators_df, influence_df, how="left", on='full_pk', suffixes=('', '_y'))
first_merge.drop(first_merge.filter(regex='_y$').columns, axis=1, inplace=True)

# print(first_merge.to_string())
print(*first_merge.columns, sep=", ")
print(*activities_df.columns, sep=", ")



full_pk, primary_key, district_code, state_abbreviation, district_string, chamber, district, state_code, chamber_code, seat_num, first_name, last_name, party, influence_score
full_pk, primary_key, first_name, last_name, activities_score, events


In [ ]:

#merge influence scores
second_merge = pd.merge(first_merge, activities_df, how="left", on='full_pk', suffixes=('', '_y'))
second_merge.drop(second_merge.filter(regex='_y$').columns, axis=1, inplace=True)

second_merge

In [7]:

# Function to count non-whitespace events
def count_events(event_str):
    if pd.isna(event_str):  # Check if the value is NaN
        return 0
    # Split by "|" and strip whitespace
    events_list = [event.strip() for event in event_str.split('|') if event.strip()]
    
    return len(events_list)

# Add a new column with the count of events
second_merge['event_count'] = second_merge['events'].apply(count_events)
# print(second_merge.columns)
# print(second_merge.to_string())



In [8]:
from datetime import date

#final exports

final_df = second_merge.copy()
print(*final_df.columns, sep=', ')


full_pk, primary_key, district_code, state_abbreviation, district_string, chamber, district, state_code, chamber_code, seat_num, first_name, last_name, party, influence_score, activities_score, events, event_count


In [9]:

for i,j in enumerate(final_df['full_pk']):
    j_pk = re.findall(r'^\d{6}', str(j))[0]
    seat_num = re.findall(r'\d{2}$', str(j))[0]
    # print(f'seat num is {seat_num}')
    # print(f'type is {type(seat_num)}')
    
    if seat_num == '00':
        # print('its a single seat')
        seat_num_v = np.nan
    else:
        seat_num_v = 'Seat ' + seat_num

    
    state_match = re.findall(r'^\d{2}', str(j_pk))
    state = state_coding_r.get(int(state_match[0]))
    chamber = int(re.findall(r'(?<=^\d{2})\d{1}(?=\d{3})', str(j_pk))[0])
    
    if chamber == 0:
        chamber_v = 'House'
    else:
        chamber_v = 'Senate'
    
    district = int(re.findall(r'(?<=^\d{3})\d{3}$', str(j_pk))[0].lstrip('0'))
    district_v = f'District {district}'
    
    
    
    if str(seat_num_v) != 'nan':
        final_df.loc[i,'seat_num'] = seat_num_v
        


    # print('################')
    # print(f'state is {state}')
    # print(f'chamber is {chamber_v}')
    # print(f'district is {district_v}')
    # print(seat_num_v)
    

    final_df.loc[i,'state'] = state
    final_df.loc[i,'chamber'] = chamber_v
    final_df.loc[i,'district'] = district_v

print(*final_df.columns, sep=', ')


final_df.loc[:,['year']] = "2024"
   

full_pk, primary_key, district_code, state_abbreviation, district_string, chamber, district, state_code, chamber_code, seat_num, first_name, last_name, party, influence_score, activities_score, events, event_count, state


In [10]:

file_name = f'compiled_scores{str(date.today()).replace('-','_')}.xlsx'
csv_file_name = f'compiled_scores{str(date.today()).replace('-','_')}.csv'
final_df.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{file_name}', index=False)
final_df.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{csv_file_name}', index=False)
final_df

,full_pk,primary_key,district_code,state_abbreviation,district_string,chamber,district,state_code,chamber_code,seat_num,first_name,last_name,party,influence_score,activities_score,events,event_count,state,year
0,10100100,101001,1,AL,Alabama State Senate District 1,Senate,District 1,10,1,0,Tim,Melson,Republican,17.0,NaN,NaN,0,AL,2024
1,10100200,101002,2,AL,Alabama State Senate District 2,Senate,District 2,10,1,0,Tom,Butler,Republican,11.0,NaN,NaN,0,AL,2024
2,10100300,101003,3,AL,Alabama State Senate District 3,Senate,District 3,10,1,0,Arthur,Orr,Republican,18.0,NaN,NaN,0,AL,2024
3,10100400,101004,4,AL,Alabama State Senate District 4,Senate,District 4,10,1,0,Garlan,Gudger,Republican,11.0,NaN,NaN,0,AL,2024
4,10100500,101005,5,AL,Alabama State Senate District 5,Senate,District 5,10,1,0,Greg,Reed,Republican,20.0,NaN,NaN,0,AL,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,57101502,571015,15,WV,West Virginia State Senate District 15,Senate,District 15,57,1,Seat 02,NaN,NaN,NaN,NaN,NaN,NaN,0,WV,2024
1990,57101601,571016,16,WV,West Virginia State Senate District 16,Senate,District 16,57,1,Seat 01,Jason,Barrett,Republican,12.0,NaN,NaN,0,WV,2024
1991,57101602,571016,16,WV,West Virginia State Senate District 16,Senate,District 16,57,1,Seat 02,Patricia,Rucker,Republican,11.0,NaN,NaN,0,WV,2024
1992,57101701,571017,17,WV,West Virginia State Senate District 17,Senate,District 17,57,1,Seat 01,Eric,Nelson,Republican,12.0,NaN,NaN,0,WV,2024


pulling in bills

In [11]:

bills_file = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\bills\leg_bills_info.xlsx"

bills = pd.read_excel(bills_file)
print(bills.columns)


Index(['primary_key', 'bill_labels', 'bill_counts'], dtype='object')


In [ ]:

compiled_plus_bills = pd.merge(final_df, bills, how="left", on='primary_key')
compiled_plus_bills


In [16]:

file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.xlsx'
csv_file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.csv'
compiled_plus_bills.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{file_name_cb}',sheet_name=f'{file_name_cb.replace(".xlsx", "")}', index=False)
compiled_plus_bills.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{csv_file_name_cb}', index=False)


In [17]:
bills_and_legislators = compiled_plus_bills.assign(
    bills=compiled_plus_bills['bill_labels'].str.split('|')  # Split the string into a list
).explode('bills')  # Create a new row for each list element

# Optionally, clean up the brackets
# bills_and_legislators['Values'] = df_expanded['Values'].str.strip('[]')
# print(compiled_plus_bills.columns)
bills_and_legislators

,full_pk,primary_key,district_code,state_abbreviation,district_string,chamber,district,state_code,chamber_code,seat_num,...,party,influence_score,activities_score,events,event_count,state,year,bill_labels,bill_counts,bills
0,10100100,101001,1,AL,Alabama State Senate District 1,Senate,District 1,10,1,0,...,Republican,17.0,NaN,NaN,0,AL,2024,NaN,NaN,NaN
1,10100200,101002,2,AL,Alabama State Senate District 2,Senate,District 2,10,1,0,...,Republican,11.0,NaN,NaN,0,AL,2024,NaN,NaN,NaN
2,10100300,101003,3,AL,Alabama State Senate District 3,Senate,District 3,10,1,0,...,Republican,18.0,NaN,NaN,0,AL,2024,S.B.55|S.B.53|S.B.98|S.B.157|S.B.56|S.J.R.61|S...,8.0,S.B.55
2,10100300,101003,3,AL,Alabama State Senate District 3,Senate,District 3,10,1,0,...,Republican,18.0,NaN,NaN,0,AL,2024,S.B.55|S.B.53|S.B.98|S.B.157|S.B.56|S.J.R.61|S...,8.0,S.B.53
2,10100300,101003,3,AL,Alabama State Senate District 3,Senate,District 3,10,1,0,...,Republican,18.0,NaN,NaN,0,AL,2024,S.B.55|S.B.53|S.B.98|S.B.157|S.B.56|S.J.R.61|S...,8.0,S.B.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,57101502,571015,15,WV,West Virginia State Senate District 15,Senate,District 15,57,1,Seat 02,...,NaN,NaN,NaN,NaN,0,WV,2024,NaN,NaN,NaN
1990,57101601,571016,16,WV,West Virginia State Senate District 16,Senate,District 16,57,1,Seat 01,...,Republican,12.0,NaN,NaN,0,WV,2024,NaN,NaN,NaN
1991,57101602,571016,16,WV,West Virginia State Senate District 16,Senate,District 16,57,1,Seat 02,...,Republican,11.0,NaN,NaN,0,WV,2024,NaN,NaN,NaN
1992,57101701,571017,17,WV,West Virginia State Senate District 17,Senate,District 17,57,1,Seat 01,...,Republican,12.0,NaN,NaN,0,WV,2024,NaN,NaN,NaN


In [18]:

file_name_bl = f'bills_and_legislators{str(date.today()).replace('-','_')}.xlsx'
csv_file_name_bl = f'bills_and_legislators{str(date.today()).replace('-','_')}.csv'
bills_and_legislators.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{file_name_bl}', sheet_name=f'{file_name_bl.replace(".xlsx", "")}', index=False)
bills_and_legislators.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{csv_file_name_bl}', index=False)


In [ ]:

# , suffixes=('', '_y'))
# second_merge.drop(second_merge.filter(regex='_y$').columns, axis=1, inplace=True)

In [ ]:



# print(final_df.columns)

# for i,j in enumerate(final_df['full_pk']):
#     print('#############')
#     print(j)




# final_df